In [2]:
import numpy as np
import pandas as pd
#from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler

In [6]:
df = pd.read_csv("../data/842entries.csv") #importing data from data folder
df.head()

/var/folders/dq/p1xrtf7j2glg2v6vbxvf650m0000gn/T/ipykernel_81740/719947893.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/842entries.csv") #importing data from data folder


,Unnamed: 0,code,npi_number,payer,price,name,url,street_address,city,state,zip_code,publish_date
0,0,842,1700979465,AETNA,14.94,Baptist Hospital,https://baptisthealthcare.pt.panaceainc.com/MR...,1000 West Moreno St,Pensacola,FL,32501,NaN
1,1,842,1700979465,BC FL,15.69,Baptist Hospital,https://baptisthealthcare.pt.panaceainc.com/MR...,1000 West Moreno St,Pensacola,FL,32501,NaN
2,2,842,1700979465,CHA HEALTH PLAN,14.94,Baptist Hospital,https://baptisthealthcare.pt.panaceainc.com/MR...,1000 West Moreno St,Pensacola,FL,32501,NaN
3,3,842,1700979465,FL COMMUNITY CARE LTC,15.99,Baptist Hospital,https://baptisthealthcare.pt.panaceainc.com/MR...,1000 West Moreno St,Pensacola,FL,32501,NaN
4,4,842,1700979465,LIGHTHOUSE MCAID PSN,15.99,Baptist Hospital,https://baptisthealthcare.pt.panaceainc.com/MR...,1000 West Moreno St,Pensacola,FL,32501,NaN


### Feature Trimming

In [7]:
#Feature Trimming
df = df.drop(columns=['Unnamed: 0', 'name', 'url', 'street_address', 'zip_code', 'publish_date'])
df.head()

,code,npi_number,payer,price,city,state
0,842,1700979465,AETNA,14.94,Pensacola,FL
1,842,1700979465,BC FL,15.69,Pensacola,FL
2,842,1700979465,CHA HEALTH PLAN,14.94,Pensacola,FL
3,842,1700979465,FL COMMUNITY CARE LTC,15.99,Pensacola,FL
4,842,1700979465,LIGHTHOUSE MCAID PSN,15.99,Pensacola,FL


In [8]:
df.dtypes

code           object
npi_number     object
payer          object
price         float64
city           object
state          object
dtype: object

### Cleaning NPIs

In [9]:
unqNPIs = df.npi_number.unique() #getting unique npis
toFix = [] #list for storing npis that need to be cleaned
string = 'check'

for npi in unqNPIs:
    if type(npi) == type(string):
        if '-' in npi:
            toFix.append(npi)

print(toFix)

['1699726406-a', '1699726406-b', '1457351389-C']


In [10]:
for ogNPI in toFix: #for each messy npi
  dashInd = ogNPI.index('-')
  cleanNPI = ogNPI[0:dashInd] #get the substring from the front to before the dash

  toChange = df.index[df['npi_number']==ogNPI].tolist() #get index of rows where npi matches original
  for j in toChange:
    df.iat[j,1] = cleanNPI #set each value to cleaned npi

In [11]:
df['npi_number'] = df['npi_number'].astype(int) #changing from string to int
df.head()

,code,npi_number,payer,price,city,state
0,842,1700979465,AETNA,14.94,Pensacola,FL
1,842,1700979465,BC FL,15.69,Pensacola,FL
2,842,1700979465,CHA HEALTH PLAN,14.94,Pensacola,FL
3,842,1700979465,FL COMMUNITY CARE LTC,15.99,Pensacola,FL
4,842,1700979465,LIGHTHOUSE MCAID PSN,15.99,Pensacola,FL


### Cleaning States

In [12]:
unqStates = df.state.unique()
#print(unqStates) #73, L states

# Deleting rows BREAKS df.index -- do not drop rows until the end

state73 = df.index[df['state']=='73'].tolist()
for k in state73:
    df.iat[k,5] = 'NV' # all 73 are from Northeastern Nevada Regional Hospital in Elko, NV

LState = df.index[df['state']=='L '].tolist() #fixing Alabama from 'L ' to 'AL'
for m in LState:
  df.iat[m,5] = 'AL'

### Cleaning Cities

In [13]:
unqCities = df.city.unique()
#print(unqCities)

df['city'] = df['city'].str.lower()

for city in unqCities: #some cities have blank space in index 0
    if type(city) == type(string): #since cannot drop null rows yet
        if(city[0] == ' '):
            changeCity = df.index[df['city']==city].tolist()
            for c in changeCity:
                df.iat[c,4] = city[1:] #remove blank space

df.head()

,code,npi_number,payer,price,city,state
0,842,1700979465,AETNA,14.94,pensacola,FL
1,842,1700979465,BC FL,15.69,pensacola,FL
2,842,1700979465,CHA HEALTH PLAN,14.94,pensacola,FL
3,842,1700979465,FL COMMUNITY CARE LTC,15.99,pensacola,FL
4,842,1700979465,LIGHTHOUSE MCAID PSN,15.99,pensacola,FL


### Cleaning Codes

In [14]:
unqCodes = df.code.unique()
#print(unqCodes) #have both dashes and commas
#some have commas before dashes > '84260,90-01'

for p in unqCodes:
    if ',' in p:
        commaIndex = p.index(',')
        cleanCode = p[0:commaIndex]
        toClean = df.index[df['code']==p].tolist() #get index of messy code (p)
                
        for q in toClean:
            df.iat[q,0] = cleanCode #set each value to cleaned (base) code

    elif '-' in p:
        dashIndex = p.index('-')
        cleanCode = p[0:dashIndex]
        toClean = df.index[df['code']==p].tolist()

        for r in toClean:
            df.iat[r,0] = cleanCode

#unqCodes = df.code.unique() #checking if codes are cleaned
#print(unqCodes)


In [15]:
df['code'] = df['code'].astype(int) #changing from string to int
df.dtypes

code            int64
npi_number      int64
payer          object
price         float64
city           object
state          object
dtype: object

### Cleaning Prices

In [30]:
badInd = [] #store indicies of bad prices (include code in price)

for row in range(len(df)): #for each row
    codeCheck = str(df.iloc[row,0]) #get code as string
    priceCheck = str(df.iloc[row,3]) #get price as string
    if codeCheck in priceCheck:
        badInd.append(row)

#print(len(badInd)) #144 rows

In [32]:
df = df.drop(index=badInd) #dropping bad rows

In [37]:
unqPrices = df.price.unique()
print(max(unqPrices)) #highest price doesn't match any codes
#print(unqPrices)

30090454.0


In [38]:
df.dtypes

code            int64
npi_number      int64
payer          object
price         float64
city           object
state          object
dtype: object

### Dropping Rows with Null Entries (must do last)

In [39]:
# Deleting rows BREAKS df.index -- do not drop rows until the end

nullPrices = df[df['price'].isnull()].index.tolist()
df = df.drop(index=nullPrices) #dropping rows with null prices

nullStates = df[df['state'].isnull()].index.tolist()
df = df.drop(index=nullStates) #dropping rows with null states

#no null city, payer, npi_numberm or code

### Ordinal Encoding

In [40]:
df.head()

,code,npi_number,payer,price,city,state
0,842,1700979465,AETNA,14.94,pensacola,FL
1,842,1700979465,BC FL,15.69,pensacola,FL
2,842,1700979465,CHA HEALTH PLAN,14.94,pensacola,FL
3,842,1700979465,FL COMMUNITY CARE LTC,15.99,pensacola,FL
4,842,1700979465,LIGHTHOUSE MCAID PSN,15.99,pensacola,FL


In [41]:
enc = OrdinalEncoder(dtype=int) #integer ordinal encoding, default is float
enc.fit(df[['payer','city', 'state']])
df[['payer','city', 'state']] = enc.transform(df[['payer','city', 'state']])
df.head()

,code,npi_number,payer,price,city,state
0,842,1700979465,107,14.94,432,8
1,842,1700979465,541,15.69,432,8
2,842,1700979465,1057,14.94,432,8
3,842,1700979465,1576,15.99,432,8
4,842,1700979465,2308,15.99,432,8


In [42]:
df = df[['code', 'npi_number', 'payer', 'city', 'state', 'price']] #reordering columns for easier split later
df.head()

,code,npi_number,payer,city,state,price
0,842,1700979465,107,432,8,14.94
1,842,1700979465,541,432,8,15.69
2,842,1700979465,1057,432,8,14.94
3,842,1700979465,1576,432,8,15.99
4,842,1700979465,2308,432,8,15.99


### Standardization / Normalization
Since the data values range greatly between features, standardizing makes it easier for the model to learn.

In [45]:
scaler = StandardScaler() # default centers mean to zero & scales to unit variance
normNumpy = scaler.fit_transform(df) #returns as numpy array

### Train / Test Split (80-20)

In [46]:
npTrain, npTest = train_test_split(normNumpy, test_size=0.2)

#convert back to pandas
pdTrain = pd.DataFrame(npTrain, columns=['code', 'npi_number', 'payer', 'city', 'state', 'price'])
pdTest = pd.DataFrame(npTest, columns=['code', 'npi_number', 'payer', 'city', 'state', 'price'])
pdTrain.head()

,code,npi_number,payer,city,state,price
0,0.018663,0.356082,-0.366220,1.441843,-0.722960,-0.005363
1,0.040282,-1.871292,0.592939,-1.022553,1.073136,-0.006521
2,0.054229,-0.066981,-0.519027,-0.319244,1.318058,-0.006850
3,-0.007836,-1.871292,0.069477,-1.022553,1.073136,-0.006793
4,0.018663,-0.066981,-0.034746,-0.319244,1.318058,-0.006545


In [47]:
yTrain = pdTrain[['price']].copy()
yTest = pdTest[['price']].copy()
yTrain.head()

,price
0,-0.005363
1,-0.006521
2,-0.006850
3,-0.006793
4,-0.006545


In [48]:
xTrain = pdTrain.drop('price', axis=1)
xTest = pdTest.drop('price', axis=1)
xTrain.head()

,code,npi_number,payer,city,state
0,0.018663,0.356082,-0.366220,1.441843,-0.722960
1,0.040282,-1.871292,0.592939,-1.022553,1.073136
2,0.054229,-0.066981,-0.519027,-0.319244,1.318058
3,-0.007836,-1.871292,0.069477,-1.022553,1.073136
4,0.018663,-0.066981,-0.034746,-0.319244,1.318058


### Downloading CSV

In [49]:
xTrain.to_csv("xTrain.csv", index = False)
yTrain.to_csv("yTrain.csv", index = False)

xTest.to_csv("xTest.csv", index = False)
yTest.to_csv("yTest.csv", index = False)